In [ ]:
import requests
from bs4 import BeautifulSoup
import concurrent.futures
import time, random

import json
import re
import os

job_links = []

# Header giả lập trình duyệt thật
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/119.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    # 👇 Tắt yêu cầu nén gzip/br
    "Accept-Encoding": "identity",
    "Accept-Language": "vi-VN,vi;q=0.9,en;q=0.8",
    "Connection": "keep-alive",
}

def crawl_page(page):
    url = f"https://www.topcv.vn/viec-lam-it?page={page}"
    try:
        # Delay ngẫu nhiên tránh bị chặn
        time.sleep(random.uniform(2, 4))

        response = requests.get(url, headers=headers, timeout=15)
        response.encoding = response.apparent_encoding  # tự đoán encoding

        # Lưu HTML của trang ra file để kiểm tra
        #with open(f"page_{page}.html", "wb") as f:
            #f.write(response.content)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            all_job = soup.find_all("div", class_="job-item-2")
            links = []
            for job in all_job:
                a_tag = job.find("a", href=True)
                if a_tag:
                    links.append(a_tag["href"])
            print(f"Trang {page}: {len(links)} links")
            return links
        else:
            print(f"Trang {page} lỗi HTTP {response.status_code}")
    except Exception as e:
        print(f"Lỗi trang {page}: {e}")
    return []


# === Chạy đa luồng nhẹ (đừng quá cao nếu không bị chặn) ===
with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    results = executor.map(crawl_page, range(2, 22))  # thử trước 20 trang
    for r in results:
        job_links.extend(r)

print("\nTổng số link thu được:", len(job_links))


Trang 3: 50 links
Trang 2: 50 links
Trang 4: 50 links
Trang 7: 50 links
Trang 6: 50 links
Trang 5: 50 links
Trang 8: 50 links
Trang 9: 50 links
Trang 10: 50 links
Trang 11: 50 links
Trang 12: 50 links
Trang 13: 50 links
Trang 14: 50 links
Trang 16: 50 links
Trang 17: 50 links
Trang 15: 50 links
Trang 18: 50 links
Trang 19: 50 links
Trang 20: 50 links
Trang 21: 50 links

Tổng số link thu được: 1000


In [62]:
job_links[0]

'https://www.topcv.vn/viec-lam/php-developer-thu-nhap-upto-25-trieu-ho-tro-an-trua-thuong-nang-suat-va-loi-nhuan-cao-tai-ha-noi/1915423.html?ta_source=ITJobs_LinkDetail'

In [63]:
# url = "https://www.topcv.vn/viec-lam/java-developer/1907639.html?ta_source=ITJobs_LinkDetail"
url = job_links[0]
#Hai link trên nó khác nhau một link của trang premium, một trang của công ty thường
#Đoạn code xử lý phía dưới đều crawl được
response = requests.get(url, headers=headers, timeout=15)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    # print(soup)

In [64]:
# title_tag = soup.find(lambda tag: tag.name in ["h2", "h3", "strong"] and "Mô tả công việc" in tag.get_text())
job_description_content = soup.find("div", class_= lambda c: c and "job-description" in c) #Mekeep, sai dấu gạch dưới và gạch giữa mà tìm mãi.
#print(title_tag) #Đã lấy được thẻ title_tag
print(job_description_content) #Đã lấy được job_description

<div class="job-description">
<div class="job-description__item">
<h3>Mô tả công việc</h3>
<div class="job-description__item--content">
<p>- Phát triển các tính năng cho các dịch vụ trong Công ty, sử dụng PHP, Laravel, MySQL</p><p>- Nghiên cứu các công nghệ lập trình mới, tiên tiến để áp dụng cải tiến hệ thống</p><p>- Thực hiện các yêu cầu từ Ban giám đốc</p>
</div>
</div>
<div class="job-description__item">
<h3>Yêu cầu ứng viên</h3>
<div class="job-description__item--content">
<p><strong>- Tốt nghiệp chuyên ngành công nghệ thông tin bậc đại học hoặc có chứng chỉ tại các trung tâm đào tạo lập trình viên (Aptech/...)</strong></p><p><strong>- Có ít nhất 2 năm kinh nghiệm làm việc với PHP và Framework Laravel</strong></p><p><strong>- Thành thạo PHP, Framework Laravel, MySQL (Đã từng làm việc với Java Springboot là điểm cộng)</strong></p><p><strong>- Thành thạo GIT, Docker</strong></p><p>- Khả năng đọc hiểu tài liệu tiếng anh tốt</p><p>- Khả năng tự nghiên cứu và áp dụng các công nghệ mới.

In [71]:
# Lấy toàn bộ nội dung text (bỏ tag HTML)
job_text = job_description_content.get_text(separator="\n", strip=True)
print(job_text)

Mô tả công việc
- Phát triển các tính năng cho các dịch vụ trong Công ty, sử dụng PHP, Laravel, MySQL
- Nghiên cứu các công nghệ lập trình mới, tiên tiến để áp dụng cải tiến hệ thống
- Thực hiện các yêu cầu từ Ban giám đốc
Yêu cầu ứng viên
- Tốt nghiệp chuyên ngành công nghệ thông tin bậc đại học hoặc có chứng chỉ tại các trung tâm đào tạo lập trình viên (Aptech/...)
- Có ít nhất 2 năm kinh nghiệm làm việc với PHP và Framework Laravel
- Thành thạo PHP, Framework Laravel, MySQL (Đã từng làm việc với Java Springboot là điểm cộng)
- Thành thạo GIT, Docker
- Khả năng đọc hiểu tài liệu tiếng anh tốt
- Khả năng tự nghiên cứu và áp dụng các công nghệ mới.
- Kỹ năng làm việc độc lập, làm việc nhóm tốt
- Trung thực, nhiệt huyết, hoà đồng
- Ưu tiên ứng viên đã từng làm việc với các công nghệ như JavaScript, HTML, CSS, Vue.js hoặc React.
Quyền lợi
- Mức lương: Up to 25M
- Review lương 1 năm một lần, không giới hạn theo năng lực
- Thưởng năng suất, thưởng lợi nhuận
- 2 ngày WFH/tháng; ít nhất 12 n

In [66]:
print("=== Tên công ty ===")
# Lấy tên công ty
company_name_label  = soup.find("div", class_ = "company-name-label")
# print(company_name_label)
if (company_name_label):
    a_tag = company_name_label.find("a")
    if a_tag:
        company_name=a_tag.get_text(strip=True)
        print(company_name)

job_detail_info = soup.find_all("div", class_="job-detail__info--section")

for section in job_detail_info:
    # Lấy tiêu đề (vd: "Mức lương", "Địa điểm", "Kinh nghiệm")
    title_tag = section.find("div", class_="job-detail__info--section-content-title")
    # Lấy giá trị (vd: "30 - 45 triệu", "Hồ Chí Minh", "4 năm")
    value_tag = section.find("div", class_="job-detail__info--section-content-value")

    # Nếu trong value có thẻ a thì lấy text bên trong a
    if value_tag:
        a_tag = value_tag.find("a")
        if a_tag:
            value_text = a_tag.get_text(strip=True)
        else:
            value_text = value_tag.get_text(strip=True)
    else:
        value_text = ""

    title_text = title_tag.get_text(strip=True) if title_tag else "Không có tiêu đề"

    # In ra kết quả
    print(f"=== {title_text} ===")
    print(value_text)


# Tìm tất cả các phần item trong mô tả
items = job_description_content.find_all("div", class_="job-description__item")

for item in items:
    title = item.find("h3").get_text(strip=True)
    content = item.find("div", class_="job-description__item--content").get_text(separator="\n", strip=True)
    print("=== ", title, " ===")
    print(content)
    print()


=== Tên công ty ===
CÔNG TY CỔ PHẦN THƯƠNG MẠI ĐIỆN TỬ BẢO KIM
=== Mức lương ===
15 - 25 triệu
=== Địa điểm ===
Hà Nội
=== Kinh nghiệm ===
2 năm
===  Mô tả công việc  ===
- Phát triển các tính năng cho các dịch vụ trong Công ty, sử dụng PHP, Laravel, MySQL
- Nghiên cứu các công nghệ lập trình mới, tiên tiến để áp dụng cải tiến hệ thống
- Thực hiện các yêu cầu từ Ban giám đốc

===  Yêu cầu ứng viên  ===
- Tốt nghiệp chuyên ngành công nghệ thông tin bậc đại học hoặc có chứng chỉ tại các trung tâm đào tạo lập trình viên (Aptech/...)
- Có ít nhất 2 năm kinh nghiệm làm việc với PHP và Framework Laravel
- Thành thạo PHP, Framework Laravel, MySQL (Đã từng làm việc với Java Springboot là điểm cộng)
- Thành thạo GIT, Docker
- Khả năng đọc hiểu tài liệu tiếng anh tốt
- Khả năng tự nghiên cứu và áp dụng các công nghệ mới.
- Kỹ năng làm việc độc lập, làm việc nhóm tốt
- Trung thực, nhiệt huyết, hoà đồng
- Ưu tiên ứng viên đã từng làm việc với các công nghệ như JavaScript, HTML, CSS, Vue.js hoặc Re

In [72]:
def extract_job_info(soup):
    output_lines = []

    company_name_label  = soup.find("div", class_="company-name-label")
    if company_name_label:
        a_tag = company_name_label.find("a")
        if a_tag:
            company_name = a_tag.get_text(strip=True)
            output_lines.append("=== Tên công ty ===")
            output_lines.append(company_name)

    job_detail_info = soup.find_all("div", class_="job-detail__info--section")
    for section in job_detail_info:
        title_tag = section.find("div", class_="job-detail__info--section-content-title")
        value_tag = section.find("div", class_="job-detail__info--section-content-value")

        if value_tag:
            a_tag = value_tag.find("a")
            if a_tag:
                value_text = a_tag.get_text(strip=True)
            else:
                value_text = value_tag.get_text(strip=True)
        else:
            value_text = ""

        title_text = title_tag.get_text(strip=True) if title_tag else "Không có tiêu đề"

        output_lines.append(f"=== {title_text} ===")
        output_lines.append(value_text)

    items = job_description_content.find_all("div", class_="job-description__item")
    for item in items:
        title = item.find("h3").get_text(strip=True)
        content = item.find("div", class_="job-description__item--content").get_text(separator="\n", strip=True)
        output_lines.append(f"=== {title} ===")
        output_lines.append(content)

    text = "\n".join(output_lines)
    # print(text)
    return text  # 🔥 thêm dòng này


In [ ]:
def text_to_json(text, filename="jobs.json"):
    # Bước 1: Tách các phần === Tiêu đề ===
    pattern = r"===\s*(.*?)\s*===\s*([\s\S]*?)(?===|$)"
    matches = re.findall(pattern, text)

    # Bước 2: Đưa vào dict
    job_info = {}
    for title, content in matches:
        clean_title = title.strip()
        clean_content = content.strip()
        job_info[clean_title] = clean_content

    # Bước 3: Đọc file hiện có (nếu có)
    if os.path.exists(filename):
        with open(filename, "r", encoding="utf-8") as f:
            try:
                data = json.load(f)
            except json.JSONDecodeError:
                data = []
    else:
        data = []

    # Bước 4: Ghi thêm (append) vào danh sách
    data.append(job_info)

    # Bước 5: Ghi lại file JSON
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f"✅ Đã thêm 1 job mới vào {filename}")
    return job_info


In [69]:
text = extract_job_info(soup)
text_to_json(text, "jobs.json")

=== Tên công ty ===
=== Tên công ty ===
CÔNG TY CỔ PHẦN THƯƠNG MẠI ĐIỆN TỬ BẢO KIM
=== Mức lương ===
15 - 25 triệu
=== Địa điểm ===
Hà Nội
=== Kinh nghiệm ===
2 năm
=== Mô tả công việc ===
- Phát triển các tính năng cho các dịch vụ trong Công ty, sử dụng PHP, Laravel, MySQL
- Nghiên cứu các công nghệ lập trình mới, tiên tiến để áp dụng cải tiến hệ thống
- Thực hiện các yêu cầu từ Ban giám đốc
=== Yêu cầu ứng viên ===
- Tốt nghiệp chuyên ngành công nghệ thông tin bậc đại học hoặc có chứng chỉ tại các trung tâm đào tạo lập trình viên (Aptech/...)
- Có ít nhất 2 năm kinh nghiệm làm việc với PHP và Framework Laravel
- Thành thạo PHP, Framework Laravel, MySQL (Đã từng làm việc với Java Springboot là điểm cộng)
- Thành thạo GIT, Docker
- Khả năng đọc hiểu tài liệu tiếng anh tốt
- Khả năng tự nghiên cứu và áp dụng các công nghệ mới.
- Kỹ năng làm việc độc lập, làm việc nhóm tốt
- Trung thực, nhiệt huyết, hoà đồng
- Ưu tiên ứng viên đã từng làm việc với các công nghệ như JavaScript, HTML, CSS,

{'Tên công ty': 'CÔNG TY CỔ PHẦN THƯƠNG MẠI ĐIỆN TỬ BẢO KIM',
 'Mức lương': '15 - 25 triệu',
 'Địa điểm': 'Hà Nội',
 'Kinh nghiệm': '2 năm',
 'Mô tả công việc': '- Phát triển các tính năng cho các dịch vụ trong Công ty, sử dụng PHP, Laravel, MySQL\n- Nghiên cứu các công nghệ lập trình mới, tiên tiến để áp dụng cải tiến hệ thống\n- Thực hiện các yêu cầu từ Ban giám đốc',
 'Yêu cầu ứng viên': '- Tốt nghiệp chuyên ngành công nghệ thông tin bậc đại học hoặc có chứng chỉ tại các trung tâm đào tạo lập trình viên (Aptech/...)\n- Có ít nhất 2 năm kinh nghiệm làm việc với PHP và Framework Laravel\n- Thành thạo PHP, Framework Laravel, MySQL (Đã từng làm việc với Java Springboot là điểm cộng)\n- Thành thạo GIT, Docker\n- Khả năng đọc hiểu tài liệu tiếng anh tốt\n- Khả năng tự nghiên cứu và áp dụng các công nghệ mới.\n- Kỹ năng làm việc độc lập, làm việc nhóm tốt\n- Trung thực, nhiệt huyết, hoà đồng\n- Ưu tiên ứng viên đã từng làm việc với các công nghệ như JavaScript, HTML, CSS, Vue.js hoặc React